# Finding large collision-free configuration-space regions with IRIS

This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/trajectories.html).  I recommend having both windows open, side-by-side!


In [ ]:
import time

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    Context,
    Diagram,
    DiagramBuilder,
    HPolyhedron,
    Hyperellipsoid,
    InverseKinematics,
    IrisInConfigurationSpace,
    IrisOptions,
    MathematicalProgram,
    MultibodyPlant,
    Parser,
    Rgba,
    RigidTransform,
    Solve,
    Sphere,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.scenarios import AddIiwa, AddWsg
from manipulation.utils import ConfigureParser

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

### Helper methods

In [ ]:
def AnimateIris(
    root_diagram: Diagram,
    root_context: Context,
    plant: MultibodyPlant,
    region: HPolyhedron,
    speed: float,
):
    """
    A simple hit-and-run-style idea for visualizing the IRIS regions:
    1. Start at the center. Pick a random direction and run to the boundary.
    2. Pick a new random direction; project it onto the current boundary, and run along it. Repeat
    """

    plant_context = plant.GetMyContextFromRoot(root_context)

    q = region.ChebyshevCenter()
    plant.SetPositions(plant_context, q)
    root_diagram.ForcedPublish(root_context)

    print("Press the 'Stop Animation' button in Meshcat to continue.")
    meshcat.AddButton("Stop Animation", "Escape")

    rng = np.random.default_rng()
    nq = plant.num_positions()
    prog = MathematicalProgram()
    qvar = prog.NewContinuousVariables(nq, "q")
    prog.AddLinearConstraint(region.A(), 0 * region.b() - np.inf, region.b(), qvar)
    cost = prog.AddLinearCost(np.ones((nq, 1)), qvar)

    while meshcat.GetButtonClicks("Stop Animation") < 1:
        direction = rng.standard_normal(nq)
        cost.evaluator().UpdateCoefficients(direction)

        result = Solve(prog)
        assert result.is_success()

        q_next = result.GetSolution(qvar)

        # Animate between q and q_next (at speed):
        # TODO: normalize step size to speed... e.g. something like
        # 20 * np.linalg.norm(q_next - q) / speed)
        for t in np.append(np.arange(0, 1, 0.05), 1):
            qs = t * q_next + (1 - t) * q
            plant.SetPositions(plant_context, qs)
            root_diagram.ForcedPublish(root_context)
            if running_as_notebook:
                time.sleep(0.05)

        q = q_next

        if not running_as_notebook:
            break

    meshcat.DeleteButton("Stop Animation")


def BuildIiwaWithShelves() -> Diagram:
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant)
    wsg = AddWsg(plant, iiwa, welded=True, sphere=False)

    parser = Parser(plant)
    ConfigureParser(parser)
    bin = parser.AddModelsFromUrl("package://manipulation/shelves.sdf")[0]
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("shelves_body", bin),
        RigidTransform([0.88, 0, 0.4]),
    )

    plant.Finalize()
    AddDefaultVisualization(builder, meshcat)

    return builder.Build()

## Using the "HOME" region as an obstacle for the "SHELF" region

If we only start IRIS with a seed point inside the shelf, then it will attempt to cover the space outside of the shelf in order to maximize volume. So we run IRIS once in the home configuration, and use that as an obstacle for the IRIS region we grow in the shelf.

In [ ]:
def home_region_as_obstacle():
    diagram = BuildIiwaWithShelves()
    context = diagram.CreateDefaultContext()
    plant = diagram.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body")

    # First seed should just be the home position.
    options = IrisOptions()
    # You'll see a few glancing collisions in the resulting region; increase this number
    # to reduce them (at the cost of IRIS running for longer)
    options.num_collision_infeasible_samples = 3 if running_as_notebook else 1
    options.random_seed = 1235
    options.require_sample_point_is_contained = True
    region = IrisInConfigurationSpace(plant, plant_context, options)

    # Add a seed for reaching into the top shelf.
    ik = InverseKinematics(plant, plant_context)
    ik.AddMinimumDistanceLowerBoundConstraint(0.001, 0.01)
    p_TopShelf = [0.95, 0, 0.65]
    grasp_constraint = ik.AddPositionConstraint(
        gripper_frame, [0, 0.1, 0], plant.world_frame(), p_TopShelf, p_TopShelf
    )

    q = ik.q()
    prog = ik.get_mutable_prog()
    prog.AddQuadraticErrorCost(np.eye(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print("IK failed")
    plant.SetPositions(plant_context, result.GetSolution(q))
    diagram.ForcedPublish(context)

    options.configuration_obstacles = [region]
    shelf_region = IrisInConfigurationSpace(plant, plant_context, options)
    AnimateIris(diagram, context, plant, shelf_region, speed=0.1)


meshcat.Delete()
home_region_as_obstacle()

## Using a "starting ellipse" covering multiple IK solutions

Instead of defining a region outside the shelf as an obstacle, we can instead call IK many times inside the shelf, and use the resulting configurations to define a local metric for the "volume" that we wish to cover with IRIS.

In [ ]:
def starting_ellipse():
    diagram = BuildIiwaWithShelves()
    context = diagram.CreateDefaultContext()
    plant = diagram.GetSubsystemByName("plant")
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body")

    # Solve IK (with collision-avoidance constraints) for each of the 8 corners inside
    # the top shelf.
    q_samples = np.zeros((plant.num_positions(), 8))
    count = 0
    for back in [True, False]:
        for right in [True, False]:
            for top in [True, False]:
                ik = InverseKinematics(plant, plant_context)
                ik.AddMinimumDistanceLowerBoundConstraint(0.001, 0.01)
                p_WTarget = np.array(
                    [
                        1.0 if back else 0.8,
                        0.23 if right else -0.23,
                        0.78 if top else 0.55,
                    ]
                )
                meshcat.SetObject(f"target{count}", Sphere(0.03), Rgba(1, 0, 0, 1))
                meshcat.SetTransform(f"target{count}", RigidTransform(p_WTarget))
                ik.AddPositionConstraint(
                    gripper_frame,
                    [0.01, 0.1, 0],
                    plant.world_frame(),
                    [0.8, -0.23, 0.55],
                    [1.04, 0.23, 0.78],
                )
                ik.AddPositionConstraint(
                    gripper_frame,
                    [-0.01, 0.1, 0],
                    plant.world_frame(),
                    [0.8, -0.23, 0.55],
                    [1.04, 0.23, 0.78],
                )
                ik.AddPositionCost(
                    gripper_frame,
                    [0, 0.1, 0],
                    plant.world_frame(),
                    p_WTarget,
                    np.eye(3),
                )
                q = ik.q()
                prog = ik.get_mutable_prog()
                prog.AddQuadraticErrorCost(0.01 * np.eye(len(q)), q0, q)
                # Don't take the solution with the base rotated the other direction
                prog.AddBoundingBoxConstraint(-0.2, 0.2, q[0])
                prog.SetInitialGuess(q, q0)
                result = Solve(ik.prog())
                # Note: IK may fail for some of the points; this is fine. We're just
                # trying to approximately define a configuration space volume.
                q_samples[:, count - 1] = result.GetSolution(q)
                plant.SetPositions(plant_context, result.GetSolution(q))
                diagram.ForcedPublish(context)
                count += 1
                # Uncomment this to see the sample points animated in Meshcat.
                # time.sleep(2)

    E = Hyperellipsoid.MinimumVolumeCircumscribedEllipsoid(q_samples)
    plant.SetPositions(plant_context, E.center())
    diagram.ForcedPublish(context)
    options = IrisOptions()
    # You'll see a few glancing collisions in the resulting region; increase this number
    # to reduce them (at the cost of IRIS running for longer)
    options.num_collision_infeasible_samples = 10 if running_as_notebook else 1
    options.random_seed = 1235
    options.starting_ellipse = E
    options.iteration_limit = 1
    region = IrisInConfigurationSpace(plant, plant_context, options)
    AnimateIris(diagram, context, plant, region, speed=0.1)


meshcat.Delete()
starting_ellipse()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1ffa96e9-51f5-40a2-b71d-52faca4ce7bc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>